A little bit of segue on the previous notebook, I've recreated another jupyter to explore more on matrix factorization for collaborative filtering without the use of TF (did gone overboard with previous notebook)

Goal in mind being using numpy and scipy for mf svd or nmf with a little bit more control on matrix factorization for different federated clients as I'll explore later.

In [50]:
from src.data_examples.ex2_data_loader import ExampleDataLoader

from enum import Enum, Flag, auto
from dataclasses import dataclass
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import svds as scipy_svds

pd.options.mode.chained_assignment = None

In [51]:
RAND_SEED = 1337
np.random.seed(RAND_SEED)

In [52]:
data_loader = ExampleDataLoader()
data_loader.download().load().clean().remap().merge()

using cached file cache\datasets\movielens\ml-100k.zip
extracting zip file content:
file already exists: data\ex2\ml-100k
file already exists: data\ex2\ml-100k\allbut.pl
file already exists: data\ex2\ml-100k\mku.sh
file already exists: data\ex2\ml-100k\README
file already exists: data\ex2\ml-100k\u.data
file already exists: data\ex2\ml-100k\u.genre
file already exists: data\ex2\ml-100k\u.info
file already exists: data\ex2\ml-100k\u.item
file already exists: data\ex2\ml-100k\u.occupation
file already exists: data\ex2\ml-100k\u.user
file already exists: data\ex2\ml-100k\u1.base
file already exists: data\ex2\ml-100k\u1.test
file already exists: data\ex2\ml-100k\u2.base
file already exists: data\ex2\ml-100k\u2.test
file already exists: data\ex2\ml-100k\u3.base
file already exists: data\ex2\ml-100k\u3.test
file already exists: data\ex2\ml-100k\u4.base
file already exists: data\ex2\ml-100k\u4.test
file already exists: data\ex2\ml-100k\u5.base
file already exists: data\ex2\ml-100k\u5.test
fil

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column user_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column movie_id - only the converter will be used.
  return func(*args, **kwargs)
c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column release_date - only the converter will be used.

In [53]:
# to trucate any other features outside of matrix factorization
# select only user_id, movie_id, and rating for our matrix

features_X = ['user_id', 'movie_id']
feature_Y = 'rating'

df_main = data_loader.df[features_X + [feature_Y]]


for feature in features_X:
  df_main[feature] = df_main[feature].astype('int32')

df_main

,user_id,movie_id,rating
0,195,241,0.2
1,304,241,0.6
2,5,241,0.4
3,233,241,0.4
4,62,241,0.2
...,...,...,...
99995,862,1678,0.2
99996,862,1677,-0.2
99997,862,1679,0.0
99998,895,1680,0.2


In [129]:
def df_to_np_sparse_matrix(df, U_c, V_c, val_c):
  # pivot the data with movie_id as column to create sparse mxn matrix 
  df = df[[U_c, V_c, val_c]]
  df_pivot = df.pivot(index=U_c, columns=V_c, values=val_c).fillna(0)
  return df_pivot.values

def sparse_matrix_normalize(mat_sparse):
  # normalize matrix from means of axis 1
  mat_mean = np.mean(mat_sparse, axis=1)
  return (mat_sparse - mat_mean.reshape(-1, 1)), mat_mean

def eval_svd_factorization(df, U_c, V_c, val_c, latent_factor=25):
  mat_sparse = df_to_np_sparse_matrix(df, U_c, V_c, val_c)
  mat_sparse_norm, mat_sparse_mean = sparse_matrix_normalize(mat_sparse)

  U, sigma, Vt = scipy_svds(mat_sparse, k=latent_factor)
  sigma = np.diag(sigma)

  mat_prediction = np.dot(np.dot(U, sigma), Vt) + mat_sparse_mean.reshape(-1, 1)

  return mat_prediction, (U, sigma, Vt)


In [94]:
mat_pred, (U, sigma, Vt) = eval_svd_factorization(df_main, 'user_id', 'movie_id', 'rating', latent_factor=25)

df_pred = pd.DataFrame(mat_pred) 
df_pred # which will directly encodes user_id as index axis 1 and movie_id as columns

(943, 1682) (943, 1682)


,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.591173,0.218883,0.216366,0.320632,0.074675,0.128291,0.496458,0.195799,0.381042,0.227432,...,0.050663,0.053200,0.050407,0.052081,0.052497,0.052149,0.052012,0.052081,0.054034,0.059397
1,0.239390,-0.008896,-0.025523,0.045449,0.015504,0.047360,0.121989,0.106183,0.207451,0.064969,...,0.012884,0.010631,0.013449,0.012604,0.011902,0.011734,0.013475,0.012604,0.013509,0.010338
2,0.042691,0.010405,0.026525,-0.016571,-0.013632,0.002582,-0.035287,-0.015781,-0.047577,0.009940,...,0.004925,0.003757,0.006038,0.005113,0.004857,0.003596,0.006630,0.005113,0.005093,0.005625
3,0.064871,-0.016956,0.025028,-0.014872,0.017362,0.010358,0.062990,-0.010724,0.058845,-0.002500,...,0.006617,0.005249,0.006122,0.006659,0.007138,0.005123,0.008194,0.006659,0.006491,0.006808
4,0.395072,0.085839,0.024492,0.187126,0.004489,0.000817,0.323427,0.142048,-0.114113,0.044969,...,0.017919,0.018186,0.017251,0.018193,0.016831,0.018249,0.018136,0.018193,0.019005,0.017480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.339086,0.015353,0.011655,0.019144,0.056034,0.024695,0.145706,0.102731,0.168634,0.023343,...,0.014449,0.012582,0.014152,0.013199,0.013107,0.014511,0.011886,0.013199,0.013769,0.011267
939,0.044876,0.006287,-0.008784,0.220981,0.060224,0.013593,0.296255,0.238947,0.194064,0.010604,...,0.018983,0.023172,0.018715,0.018549,0.019140,0.017507,0.019591,0.018549,0.018872,0.017431
940,0.262788,0.015168,0.027576,0.006428,-0.001998,0.005935,0.189606,0.089612,0.125642,0.024956,...,0.004565,0.003910,0.005605,0.005351,0.006312,0.005493,0.005208,0.005351,0.006143,0.005221
941,0.135355,0.030077,-0.000674,0.020363,0.003278,0.003302,0.000714,0.173069,0.007655,0.024929,...,0.022885,0.023883,0.022420,0.021284,0.021475,0.019655,0.022914,0.021284,0.022923,0.020821


In [95]:
sparse_df_main = df_to_np_sparse_matrix(df_main, 'user_id', 'movie_id', 'rating')
# sparse_df_main = csr_array((df_main['rating'].values, (df_main['user_id'].values, df_main['movie_id'].values))).toarray()
sparse_df_pred = df_pred.to_numpy()


mse = np.mean((sparse_df_main - sparse_df_pred)**2)
rmse = np.sqrt(mse)

print(f'centralized model -> rmse: {rmse:05f}\tmse: {mse:05f}')
sparse_df_main

centralized model -> rmse: 0.072413	mse: 0.005244


array([[0.6, 0.2, 0.4, ..., 0. , 0. , 0. ],
       [0.4, 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0.6, 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0.6, 0. , ..., 0. , 0. , 0. ]])

In [58]:
tgt_user_id = 420
n_movies = 15

pred_movie_ids = df_pred.iloc[tgt_user_id].sort_values(ascending=False)[:n_movies].index

# a movie string formatter, note: uses dataloader.df_movies
fmt_movie_printout = lambda movie_id: f'movie_id: {movie_id}'.ljust(17) + data_loader.df_movies.loc[movie_id]['title']

pred_movie_ids = df_main[df_main['user_id'] == tgt_user_id]['movie_id'] [:n_movies]
print('recommendtion')
for movie_id in pred_movie_ids:
  print(fmt_movie_printout(movie_id))

pred_movie_ids


recommendtion
movie_id: 237    Raising Arizona (1987)
movie_id: 172    Princess Bride, The (1987)
movie_id: 268    Full Monty, The (1997)
movie_id: 301    L.A. Confidential (1997)
movie_id: 332    Game, The (1997)
movie_id: 116    Rock, The (1996)
movie_id: 99     Fargo (1996)
movie_id: 78     Fugitive, The (1993)
movie_id: 97     Silence of the Lambs, The (1991)
movie_id: 55     Pulp Fiction (1994)
movie_id: 11     Usual Suspects, The (1995)
movie_id: 330    Edge, The (1997)
movie_id: 95     Terminator 2: Judgment Day (1991)
movie_id: 175    Aliens (1986)
movie_id: 207    Young Frankenstein (1974)


1007     237
5660     172
6192     268
6523     301
8476     332
10247    116
12017     99
14192     78
15887     97
16276     55
18593     11
19589    330
21931     95
22948    175
24126    207
Name: movie_id, dtype: int32

## Federated Learning with Clients (or Silos) Manager

In [111]:
global_latent_factor = 25

In [112]:
class DBKind(Enum):
  shared = auto()
  private = auto()

In [113]:
class _TaskFlag(Flag):
  RUN_ONCE = auto()
  _ = auto()

class _TaskState(Enum):
  stopped = auto()
  running = auto()
  finished = auto()


class TaskWrapper:
  def __init__(self, name, fn_task: callable = None):
    self.flags = _TaskFlag._
    self.state = _TaskState.stopped
    self.fn_task = fn_task

  def set_fn(self, fn_task):
    self.fn_task = fn_task

  def run(self, *args, **kwargs):
    if self.fn_task is None:
      return

    if _TaskFlag.RUN_ONCE in self.flags and \
    self.state == _TaskState.finished:
      return

    self.state = _TaskState.running
    r = self.fn_task(*args, **kwargs)
    self.state = _TaskState.finished
    return r

  def reset(self):
    self.state = _TaskState.stopped

In [114]:
# crude emulation server-clients with ClientManager

class Client:
  def __init__(self, _uuid=None):
    self.__task_pool = []
    self.__private_dbs = {}
    self.__shared_dbs = {}
    self.__named_variables = {}
    if _uuid is None:
      self._uuid = id(self) # fallbacks to basically the address on memory of client object for the sake of simplicity
    else:
      self._uuid = _uuid

  # client uuid
  def get_uuid(self):
    return self._uuid

  # client tasks
  def clear_tasks(self):
    self.__task_pool.clear()

  def add_task(self, fn):
    self.__task_pool.append(fn)

  def run_tasks(self):
    for task in self.__task_pool:
      if isinstance(task, TaskWrapper):
        task.run(self)
      else:
        task(self)

  # client public named variables 
  def clear_named_variables(self):
    self.__named_variables.clear()

  def set_named_variable(self, name, value):
    self.__named_variables[name] = value

  def get_named_variable_value(self, name):
    return self.__named_variables.get(name)

  def get_named_variable_value_names(self):
    return self.__named_variables.keys()

  # client database connection (read: dataframe)
  def add_db(self, kind: DBKind, name, db):
    if kind == DBKind.shared:
      self.__shared_dbs[name] = db
    elif kind == DBKind.private:
      self.__private_dbs[name] = db

  def get_db(self, kind: DBKind, name):
    if kind == DBKind.shared:
      return self.__shared_dbs[name]
    elif kind == DBKind.private:
      return self.__private_dbs[name]


class ClientManager:
  def __init__(self):
    self._client_bucket = []
    self._client_uuids = []

  def new_client(self, _uuid=None):
    client = Client(_uuid=_uuid)
    self._client_bucket.append(client)
    self._client_uuids.append(client.get_uuid())
    if _uuid is None:
      client._uuid = id(client)
    else:
      client._uuid = _uuid
    return client

  def get_client_uuids(self):
    return self._client_uuids

  def get_client_by_uuid(self, _uuid):
    if _uuid not in self._client_uuids:
      return None
    return self._client_bucket[self._client_uuids.index(_uuid)]

  def force_run_tasks(self):
    for client in self._client_bucket:
      client.run_tasks()

  def force_clear_tasks(self):
    for client in self._client_bucket:
      client.clear_tasks()
    
  def force_clear_named_variabless(self):
    for client in self._client_bucket:
      client.clear_named_variables()

  def run_callback_on_clients(self, fn: callable, *args, **kwargs):
    for client in self._client_bucket:
      fn(client, *args, **kwargs)


# example
manager = ClientManager()
for _ in range(4):
  client = manager.new_client()
  task = TaskWrapper('print stdout')
  task.set_fn(lambda this: print(f'running task on client: printing uuid {this.get_uuid()}'))
  client.add_task(task)

print('client uuids:', manager.get_client_uuids())
manager.force_run_tasks()

client uuids: [1774738654400, 1774738652992, 1774738652080, 1774738650976]
running task on client: printing uuid 1774738654400
running task on client: printing uuid 1774738652992
running task on client: printing uuid 1774738652080
running task on client: printing uuid 1774738650976


### try federated learining from clusters of zipcodes

In [115]:
n_zip_code_clusters = 20

unique_zip_codes = data_loader.df_users['zip_code'].unique()
np.random.shuffle(unique_zip_codes)
unique_zip_codes_clusters = np.array_split(unique_zip_codes, n_zip_code_clusters)


C:\Users\62896\AppData\Local\Temp\ipykernel_3932\2911003581.py:4: UserWarning: you are shuffling a 'StringArray' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(unique_zip_codes)


In [116]:
user_ratings_headers = ['zip_code', 'user_id', 'movie_id', 'rating']

manager = ClientManager()

for zip_codes in unique_zip_codes_clusters:
  uuid_name = '-'.join(zip_codes[:3]) + '...'
  client_ref = manager.new_client(_uuid=uuid_name)

  client_ref.add_db(kind=DBKind.shared, name='df_movies', db=data_loader.df_movies)

  zip_code_mask = data_loader.df['zip_code'].apply(lambda x: x in zip_codes)
  private_df = data_loader.df[zip_code_mask]
  private_df = private_df[user_ratings_headers] # .copy()
  client_ref.add_db(kind=DBKind.private, name='df_user_ratings', db=private_df.sample(frac=1))


In [117]:
# inspec one of the client private user->ratings dataset

__client_ref = manager.get_client_by_uuid(manager.get_client_uuids()[0])
__client_ref.get_db(DBKind.private, 'df_user_ratings')

,zip_code,user_id,movie_id,rating
39395,44133,82,507,0.0
77965,21201,751,538,0.4
29617,22902,536,522,0.2
24436,22902,536,171,0.2
34524,95403,737,173,0.6
...,...,...,...,...
34420,93711,473,173,0.6
18215,21201,751,886,-0.2
22880,63033,214,175,0.6
76923,55346,262,494,0.6


### adding svd fit tasks to each clients

In [130]:
# force clear task to prevent flooding of print statement or dupicate tasks in pool
manager.force_clear_tasks()

In [131]:
for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)

  task = TaskWrapper('runtime log')
  task.flags |= _TaskFlag.RUN_ONCE
  task.set_fn(
    lambda client: print(f'running task on client "{client.get_uuid()}" with num of participating users: {client.get_db(DBKind.private, "df_user_ratings")["user_id"].unique().shape[0]}')
  )

  client_ref.add_task(task)

  client_ref.set_named_variable('latent_factor', global_latent_factor)

# try to run newly created print tasks on client
manager.force_run_tasks()


running task on client "06492-05201-21201..." with num of participating users: 46
running task on client "06405-53144-77081..." with num of participating users: 53
running task on client "V5A2B-33755-43215..." with num of participating users: 47
running task on client "90019-29631-52245..." with num of participating users: 42
running task on client "61401-04102-55117..." with num of participating users: 52
running task on client "20008-63304-95014..." with num of participating users: 45
running task on client "97330-23226-75240..." with num of participating users: 45
running task on client "77841-70403-31909..." with num of participating users: 51
running task on client "91505-12550-60515..." with num of participating users: 52
running task on client "78750-49428-37777..." with num of participating users: 42
running task on client "63129-11753-76111..." with num of participating users: 46
running task on client "49036-92110-55413..." with num of participating users: 49
running task on 

In [132]:
def task_eval_svd_factorization(client):
  print('begin factorization ...', end=' ')

  df = client.get_db(DBKind.private, 'df_user_ratings')
  latent_factor = client.get_named_variable_value('latent_factor')

  mat_pred, (U, sigma, Vt) = eval_svd_factorization(df, 'user_id', 'movie_id', 'rating', latent_factor=latent_factor)

  client.set_named_variable('local_prediction_matrix', mat_pred)
  client.set_named_variable('U', U)
  client.set_named_variable('sigma', sigma)
  client.set_named_variable('Vt', Vt)

  print('finished factorization\n')


for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)
  task = TaskWrapper('svd factorization')
  task.flags |= _TaskFlag.RUN_ONCE
  task.set_fn(task_eval_svd_factorization)
  client_ref.add_task(task)

In [133]:
# run all factorization tasks
manager.force_run_tasks()

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization



begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization

begin factorization ... finished factorization



In [134]:
# fetch all client saved variables as weakrefs, stored in 'server' as dict() notebook local variable in this case

clients_named_variables = {}
for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)

  clients_named_variables[client_uuid] = {}

  for key in ['sigma']: # 'U', 'Vt'
    val = client_ref.get_named_variable_value(key)
    clients_named_variables[client_uuid][key] = val

In [135]:
# compiling sigma diag matrixes with averatge aggregation based on number of clients

sigmas = [i['sigma'] for i in clients_named_variables.values()]

sigma_avg = sigmas.pop(0)
for sig in sigmas:
  sigma_avg += sig

sigma_avg /= (len(sigmas) + 1)


# ship or broadcast aggregated sigma values to clients

for client_uuid in manager.get_client_uuids():
  client_ref = manager.get_client_by_uuid(client_uuid)
  client_ref.set_named_variable('sigma_agg', sigma_avg)


# inspec aggregated sigma values
sigma_avg, sigma_avg.shape

(array([[ 2.70902498,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  2.80113973,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  2.86157418,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  

### perform inference with first instance of user on each cluster/client  

In [139]:
def callback_metrics(client):
  df_local = client.get_db(DBKind.private, 'df_user_ratings')

  sparse_local = df_to_np_sparse_matrix(df_local, 'user_id', 'movie_id', 'rating')

  U = client.get_named_variable_value('U')
  sigma = client.get_named_variable_value('sigma_agg')
  Vt = client.get_named_variable_value('Vt')

  sparse_mat_pred_local = np.dot(np.dot(U, sigma), Vt) 

  mse = np.mean((sparse_local - sparse_mat_pred_local)**2)
  rmse = np.sqrt(mse)

  print(f'federated cluster model -> rmse: {rmse:05f}\tmse: {mse:05f}')
manager.run_callback_on_clients(callback_metrics)

federated cluster model -> rmse: 0.047939	mse: 0.002298
federated cluster model -> rmse: 0.046922	mse: 0.002202
federated cluster model -> rmse: 0.038910	mse: 0.001514
federated cluster model -> rmse: 0.036619	mse: 0.001341
federated cluster model -> rmse: 0.048609	mse: 0.002363
federated cluster model -> rmse: 0.040251	mse: 0.001620
federated cluster model -> rmse: 0.037121	mse: 0.001378
federated cluster model -> rmse: 0.044466	mse: 0.001977
federated cluster model -> rmse: 0.045945	mse: 0.002111
federated cluster model -> rmse: 0.046092	mse: 0.002124
federated cluster model -> rmse: 0.040457	mse: 0.001637
federated cluster model -> rmse: 0.039813	mse: 0.001585
federated cluster model -> rmse: 0.042424	mse: 0.001800
federated cluster model -> rmse: 0.036310	mse: 0.001318
federated cluster model -> rmse: 0.037710	mse: 0.001422
federated cluster model -> rmse: 0.046159	mse: 0.002131
federated cluster model -> rmse: 0.039637	mse: 0.001571
federated cluster model -> rmse: 0.047656	mse: 0

In [161]:
def callback_predict(client):
  n_movies = 3
  df_user = client.get_db(DBKind.private, 'df_user_ratings')['user_id'] # for now uses first instance of the user on each client
  user_ls = df_user.unique().tolist()
  user = df_user.iloc[0]
  user_idx = user_ls.index(user)
  
  print('getting recommendation for user:', f'{user}', f', in cluster "{client.get_uuid()}"')

  U = client.get_named_variable_value('U')
  sigma = client.get_named_variable_value('sigma_agg')
  Vt = client.get_named_variable_value('Vt')

  new_local_mat_pred = np.dot(np.dot(U, sigma), Vt) 

  df_pred = pd.DataFrame(new_local_mat_pred)
  
  pred_movie_ids = df_pred.loc[user_idx, :].sort_values(ascending=False)[:n_movies].index

  for n, movie_id in enumerate(pred_movie_ids):
    print(f'No.{n+1}'.ljust(5), fmt_movie_printout(movie_id))
  print()

manager.run_callback_on_clients(callback_predict)

getting recommendation for user: 82 , in cluster "06492-05201-21201..."
No.1  movie_id: 780    French Kiss (1995)
No.2  movie_id: 327    Conspiracy Theory (1997)
No.3  movie_id: 430    Highlander (1986)

getting recommendation for user: 654 , in cluster "06405-53144-77081..."
No.1  movie_id: 942    Killing Zoe (1994)
No.2  movie_id: 538    Mouse Hunt (1997)
No.3  movie_id: 575    Cliffhanger (1993)

getting recommendation for user: 371 , in cluster "V5A2B-33755-43215..."
No.1  movie_id: 591    True Crime (1995)
No.2  movie_id: 502    Candidate, The (1972)
No.3  movie_id: 385    Addams Family Values (1993)

getting recommendation for user: 899 , in cluster "90019-29631-52245..."
No.1  movie_id: 351    Spice World (1997)
No.2  movie_id: 262    Steel (1997)
No.3  movie_id: 274    Sense and Sensibility (1995)

getting recommendation for user: 182 , in cluster "61401-04102-55117..."
No.1  movie_id: 429    Duck Soup (1933)
No.2  movie_id: 417    Cinderella (1950)
No.3  movie_id: 430    Highl